# Speed Skydiving Analysis and Scoring 2019

Analyze one or more FlySight files with speed skydiving data.

This document implements scoring techniques compatible with the FAI World Air Sports Federation [Speed Skydiving Competition Rules, 2019 Edition](https://www.fai.org/sites/default/files/documents/2019_ipc_cr_speedskydiving.pdf) (PDF, 428 KB).

## Environment setup

In [1]:
from ssscore import COURSE_END
from ssscore import DEG_IN_RAD
from ssscore import VALID_MSL

import dateutil.parser
import inspect
import math
import os
import os.path
import shutil

import pandas as pd

import ssscore

### Known drop zones AMSL in meters

The `ssscore` module defines these altitudes; the DZ name corresponds to the symbolic constant, e.g. Bay Area Skydiving ::= `BAY_AREA_SKYDIVING`.  The altitudes were culled from public airport information available on the Worldwide Web.

In [2]:
import ssscore.elevations as elevations # expressed in meters

print(inspect.getsource(elevations))

# vim: set fileencoding=utf-8:

# BSD 3-Clause License -- see ../../LICENSE for details.


# TODO: Define an enum or similar object, to express the altitude in ft, m?
#       Keep these definitions and erase this TODO if this comment is still here
#       after 30.Jun.2019

# Drop zone name, altitude in meters

BAY_AREA_SKYDIVING           =   23.90
CHICAGOLAND_SKYDIVING_CENTER =  238.00
SKYDANCE_SKYDIVING           =   30.48
SKYDIVE_ALGARVE              =    2.00
SKYDIVE_ARIZONA              =  460.60
SKYDIVE_BUZZ                 =  256.00
SKYDIVE_FANO                 =   54.00
SKYDIVE_FLANDERS_ZWARTBERG   =   85.00
SKYDIVE_SAULGAU              = 1076.00
SKYDIVE_UTAH                 = 1317.04




#### Set the appropriate DZ elevation

In [3]:
DZ_AMSL = elevations.SKYDANCE_SKYDIVING

## FlySight data sources

1. Copy the FlySight `Tracks/YY-MM-dd` directory of interest to the `DATA_LAKE` directory;
   the `DATA_LAKE` can also be an external mount, a Box or Dropbox share, anything that 
   can be mapped to a directory -- even a whole drive!
1. Make a list of all CSV files in the FlySight `DATA_LAKE`
1. Move the CSV to the `DATA_SOURCE` bucket directory

### Define the data lake and data source

![Data sources diagram](images/SSScoring-data-sources.png)
<a id="l_data-def"></a>

In [4]:
DATA_LAKE   = os.path.join('.', ssscore.RESOURCE_PATH)
DATA_SOURCE = os.path.join('.', 'data-sources', ssscore.RESOURCE_PATH)

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

moved ./resources/FlySight-test-file.csv --> ./data-sources/resources/FlySight-test-file.csv
moved 1 CSV files from data lake [./resources] to data source [./data-sources/resources]


## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Maximum mean speed within a 3-second interval within the course
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the exit altitude to 1,700 m AGL course
1. Resolve the max speed and pitch within the course
1. Calculate the max mean speed for the 3-second interval near the max speed
1. Return the results in a Series object for later inclusion in a data frame with results from multiple jumps

### Specify the FlySight data file to analyze

In [5]:
DATA_SOURCE        = os.path.join('.', 'data-sources', ssscore.RESOURCE_PATH)
DATA_SOURCE        = './data-sources/ciurana'
FLYSIGHT_DATA_FILE = 'FlySight-test-file.csv'
FLYSIGHT_DATA_FILE = '22-20-00.CSV'

In [6]:
def _discardClimbAltitudeDropsIn(flightData):
    """
        Altitude drops in flightData occur when a load includes hop and pops, or when the pilot decides to
        circle or descend for safety reasons.  This function discards all minor altitude drops and keeps
        only the speed skydive course.
        
        Accepts:  flight data
        Returns:  the flight data corresonding to the speed skydive course
    """
    courseData = flightData.copy()
    courseData['descends', 'Boolean'] = courseData['velD', '(m/s)'] >= 0.0
    courseData['group', ''] = (courseData['descends', 'Boolean'] != courseData['descends', 'Boolean'].shift(1)).cumsum()
    
    courseGroup = courseData['group', ''].max()
    
    return courseData[courseData['group', ''] == courseGroup]

In [7]:
def _discardDataOutsideCourse(flightData):
    maxHeight       = flightData['hAGL', '(m)'].max()
    height          = flightData['hAGL', '(m)']
    descentVelocity = flightData['velD', '(m/s)']
    flightData      = flightData[(height <= maxHeight) & (height >= COURSE_END) & (descentVelocity >= 0.0)]
    
    return _discardClimbAltitudeDropsIn(flightData)

In [8]:
def _calculateCourseSpeedUsing(flightData):
    """
    Returns absolute best max speed and 3-second window max speed.
    """
    # TODO: figure out why pd.to_datetime() barfs
    startTime = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[0])
    endTime   = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[-1])

    startCourse = flightData['hAGL', '(m)'].values[0]
    endCourse   = flightData['hAGL', '(m)'].values[-1]

    return flightData['velD', '(m/s)'].max(), 0.0, startCourse, endCourse

In [9]:
def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector

In [10]:
"""
    Adjusts the flight data to compensate for DZ elevation AMSL.
    
    flightData - the raw FlySight data frame
    elevation  - the elevation, in meters, to adjust
    
    All hMSL values in flightData will be offset by +elevation meters.
"""
def adjustElevation(flightData, elevation):
    flightData['hAGL', '(m)'] = flightData['hMSL', '(m)']-elevation
    
    return flightData

In [13]:
def calculateSpeedAndPitchFor(fileName, elevation = 0.00):
    """
    Accepts a file name to a FlySight data file.
    
    Returns a Series with the results of a speed skydiving jump.
    """
    flightData  = adjustElevation(pd.read_csv(fileName, header = [0, 1]), elevation)
    flightData  = _discardDataOutsideCourse(flightData)

    maxSpeed, \
    bestSpeed, \
    startCourse, \
    endCourse = _calculateCourseSpeedUsing(flightData)
    
    pitchR       = math.atan(maxSpeed/maxHorizontalSpeedFrom(flightData, maxSpeed))
    # maxSpeedTime = flightData[flightData['velD', '(m/s)'] == maxSpeed]['time', 'Unnamed: 0_level_1'].values[0]
    
    skydiveResults = pd.Series(
                        [
                            fileName,
                            3.6*maxSpeed,  # km/h; 3,600 seconds, 1,000 meters
                            3.6*bestSpeed,
                            pitchR/DEG_IN_RAD,
                            startCourse,
                            endCourse,
                        ],
                        [
                            'fileName',
                            'maxSpeed',
                            'bestSpeed',
                            'pitch',
                            'startCourse',
                            'endCourse',

                        ])

    return skydiveResults
#     flightData.to_csv('./data-sources/test.csv')
#     return flightData


calculateSpeedAndPitchFor(os.path.join(DATA_SOURCE, FLYSIGHT_DATA_FILE), elevation = DZ_AMSL)

fileName       ./data-sources/ciurana/22-20-00.CSV
maxSpeed                                   381.924
bestSpeed                                        0
pitch                                      67.7972
startCourse                                1762.13
endCourse                                   1712.3
dtype: object

## Listing FlySight data files with valid data

This set up generates a list of FlySight data files ready for analysis.  It discards any warm up FlySight data files, those that show no significant changes in elevation MSL across the complete data set.

The list generator uses the `DATA_SOURCE` global variable.  [Change the value of `DATA_SOURCE`](#l_data-def) if necessary.

1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

![Scoring FlySight files list generation diagram](images/SSScoring-list-scoring-files.png)

In [14]:
def listDataFilesIn(bucketPath):
    """
    Generate a sorted list of files available in a given bucketPath.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(bucketPath, fileName) for fileName in sorted(os.listdir(bucketPath), reverse = True) if '.CSV' in fileName or '.csv' in fileName])
    
    return filesList;

In [15]:
def _hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [16]:
def selectValidFlySightFilesFrom(dataFiles):
    included = dataFiles.apply(_hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [17]:
dataFiles = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles

0    ./data-sources/ciurana/23-17-01.CSV
1    ./data-sources/ciurana/22-20-00.CSV
2    ./data-sources/ciurana/21-05-17.CSV
3    ./data-sources/ciurana/20-03-57.CSV
dtype: object

## Top speed and pitch analysis on all tracks in the data lake

Takes all the FlySight files in a bucket, detects the ones with valid data, and runs performance analysis over them.  Packs all the results in a data frame, then calculates:

* Average speed
* Max average speed

### Populate data sources

In [18]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/ciurana'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

moved 0 CSV files from data lake [./data-lake] to data source [./data-sources/ciurana]


In [32]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/landgren'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

moved 0 CSV files from data lake [./data-lake] to data source [./data-sources/landgren]


### Analyze all files in the bucket

In [19]:
allCompetitionJumps = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE)).apply(calculateSpeedAndPitchFor)
allCompetitionJumps

,fileName,maxSpeed,bestSpeed,pitch,startCourse,endCourse
0,./data-sources/ciurana/23-17-01.CSV,377.352,0.0,71.620933,4164.380,1707.171
1,./data-sources/ciurana/22-20-00.CSV,381.924,0.0,67.797208,1792.609,1706.337
2,./data-sources/ciurana/21-05-17.CSV,380.052,0.0,71.623893,1736.275,1702.276
3,./data-sources/ciurana/20-03-57.CSV,364.104,0.0,64.330662,4110.713,1714.890


### Summary of results

In [20]:
summary = pd.Series(
            [
                len(allCompetitionJumps),
                allCompetitionJumps['maxSpeed'].mean(),
                allCompetitionJumps['pitch'].mean(),
                allCompetitionJumps['maxSpeed'].max(),
                allCompetitionJumps['pitch'].max(),
            ],
            [
                'totalJumps',
                'meanSpeed',
                'pitch',
                'maxSpeed',
                'maxPitch',
            ])

summary

totalJumps      4.000000
meanSpeed     375.858000
pitch          68.843174
maxSpeed      381.924000
maxPitch       71.623893
dtype: float64